# 📊 Desafio de Código – Pipeline ETL em Python

Este projeto tem como objetivo demonstrar, de forma prática, a aplicação dos conceitos e boas práticas de **ETL (Extract, Transform, Load)** utilizando Python. O pipeline foi desenvolvido a partir de dados públicos do **Portal da Transparência**, preparando o dataset final para análise no **Power BI**.

A solução foi estruturada de maneira modular e organizada, facilitando a leitura, manutenção e reaproveitamento do código, além de evidenciar a capacidade técnica aplicada na resolução do desafio.

# ⚙️ Etapa 0 – Configuração do Ambiente

Nesta etapa inicial, o ambiente é preparado para garantir organização e reprodutibilidade do processo:

Importação de bibliotecas padrão do Python (`os`, `zipfile`, `glob`, `unicodedata`) e bibliotecas externas amplamente utilizadas em projetos de dados (`requests` e `pandas`);

Definição de diretórios separados para **dados brutos** (`data_raw`) e **dados processados** (`data_processed`), seguindo boas práticas de organização;

Criação automática dessas pastas, evitando erros em execuções futuras.

Essa estrutura facilita tanto o entendimento do fluxo quanto a escalabilidade do projeto.

In [1]:
# ============================================================
# ETAPA 0 – CONFIGURAÇÃO DO AMBIENTE
# ============================================================

# Bibliotecas padrão
import os
import zipfile
import glob
import unicodedata

# Bibliotecas externas
import requests
import pandas as pd

# 📥 Etapa 1 – Extração dos Dados (Extract)

A etapa de extração é responsável por obter os dados diretamente da fonte oficial:

* **Download automatizado** do arquivo ZIP a partir do Portal da Transparência, utilizando requisições HTTP com tratamento de erros;

* **Armazenamento do arquivo bruto localmente**, garantindo rastreabilidade dos dados originais;

* **Extração controlada do conteúdo do ZIP**, evitando duplicidades;

* **Identificação automática do arquivo CSV** para processamento.

Essa abordagem garante **confiabilidade** na origem dos dados e reduz dependências manuais.


In [5]:
# ============================================================
# ETAPA 1 – EXTRAÇÃO DOS DADOS
# ============================================================

URL = "https://portaldatransparencia.gov.br/download-de-dados/despesas-favorecidos/202511"

try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    BASE_DIR = os.getcwd()

RAW_DIR = os.path.join(BASE_DIR, "data_raw")
PROCESSED_DIR = os.path.join(BASE_DIR, "data_processed")

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(URL, headers=headers)

if response.status_code != 200:
    raise Exception(f"Erro no download | Status: {response.status_code}")

zip_path = os.path.join(RAW_DIR, "despesas_favorecidos_202511.zip")

with open(zip_path, "wb") as f:
    f.write(response.content)

print("📦 ZIP baixado com sucesso!")

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    for member in zip_ref.namelist():
        target_path = os.path.join(RAW_DIR, member)
        if os.path.exists(target_path):
            os.remove(target_path)
        zip_ref.extract(member, RAW_DIR)

print("📂 ZIP extraído com sucesso!")

csv_files = glob.glob(os.path.join(RAW_DIR, "*.csv"))

if not csv_files:
    raise Exception("Nenhum arquivo CSV encontrado após extração do ZIP.")

raw_file = csv_files[0]
print(f"📄 CSV identificado: {os.path.basename(raw_file)}")


📦 ZIP baixado com sucesso!
📂 ZIP extraído com sucesso!
📄 CSV identificado: 202511_RecebimentosRecursosPorFavorecido.csv


# 🔄 Etapa 2 – Transformação dos Dados (Transform)

Esta é a etapa central do processo, onde os dados brutos são tratados e padronizados para uso analítico.

## 2.1 Padronização dos nomes das colunas

Os nomes das colunas são normalizados para remover acentos, espaços e inconsistências de formatação, facilitando o uso em ferramentas analíticas e evitando erros futuros.

## 2.2 Conversão de valores monetários

A coluna de valores financeiros é convertida corretamente para o tipo numérico, respeitando o padrão brasileiro de separadores decimais, garantindo precisão em cálculos e análises.

## 2.3 Tratamento de datas

A coluna de data é convertida para o formato de data do Python, com tratamento de valores inválidos, assegurando consistência temporal no dataset.

## 2.4 Conversão de códigos numéricos

Colunas que representam códigos administrativos são convertidas para o tipo inteiro, preservando valores nulos quando necessário e garantindo melhor desempenho em análises e relacionamentos.

## 2.5 Padronização de colunas de texto

As colunas textuais passam por limpeza básica, como remoção de espaços extras, garantindo maior qualidade dos dados.

### 🔍 Destaque importante:
A coluna **CodigoFavorecido** foi mantida e tratada como **texto**, pois na base original ela pode conter **valores alfanuméricos**. Essa decisão evita perda de informação e demonstra atenção à natureza real dos dados, uma prática essencial em projetos de dados profissionais.


In [6]:
# ============================================================
# ETAPA 2 – TRANSFORMAÇÃO DOS DADOS
# ============================================================

df = pd.read_csv(
    raw_file,
    sep=';',
    encoding='latin1',
    low_memory=False
)

# ------------------------------------------------------------
# 2.1 – Padronização dos nomes das colunas
# ------------------------------------------------------------

def normalizar_coluna(col):
    col = col.strip()
    col = unicodedata.normalize("NFKD", col).encode("ascii", "ignore").decode("utf-8")
    col = col.replace(" ", "")
    return col

df.columns = [normalizar_coluna(c) for c in df.columns]

if "Anoemesdolancamento" in df.columns:
    df.rename(columns={"Anoemesdolancamento": "DataLancamento"}, inplace=True)
else:
    raise Exception("Coluna de data não encontrada no dataset.")

# ------------------------------------------------------------
# 2.2 – Conversão de valores monetários
# ------------------------------------------------------------

df["ValorRecebido"] = (
    df["ValorRecebido"]
      .astype(str)
      .str.replace(".", "", regex=False)
      .str.replace(",", ".", regex=False)
      .astype(float)
)

# ------------------------------------------------------------
# 2.3 – Conversão da coluna de data
# ------------------------------------------------------------

df["DataLancamento"] = pd.to_datetime(
    df["DataLancamento"],
    dayfirst=True,
    errors="coerce"
)

# ------------------------------------------------------------
# 2.4 – Conversão de códigos para inteiro
# ------------------------------------------------------------

colunas_inteiro = [
    "CodigoOrgaoSuperior",
    "CodigoOrgao",
    "CodigoUnidadeGestora"
]

for col in colunas_inteiro:
    df[col] = (
        df[col]
          .astype(str)
          .str.replace(".0", "", regex=False)
          .replace("nan", None)
          .astype("Int64")
    )

# ------------------------------------------------------------
# 2.5 – Padronização das colunas texto
# ------------------------------------------------------------

colunas_texto = [
    "CodigoFavorecido",
    "NomeFavorecido",
    "SiglaUF",
    "NomeMunicipio",
    "NomeOrgaoSuperior",
    "NomeOrgao",
    "NomeUnidadeGestora"
]

for col in colunas_texto:
    df[col] = df[col].astype(str).str.strip()


# 📈 Etapa 3 – Preparação Final para Análise (Load)

Na etapa final, os dados são preparados para consumo no Power BI:

Garantia dos tipos finais das colunas, assegurando compatibilidade com ferramentas de BI;

Seleção e ordenação das colunas mais relevantes para análise;

Exportação do dataset final em formato CSV, com encoding adequado e separador compatível com o Power BI.

O arquivo final é salvo na pasta de dados processados, pronto para visualizações e análises.

In [7]:
# ============================================================
# ETAPA 3 – PREPARAÇÃO FINAL PARA POWER BI
# ============================================================

# ------------------------------------------------------------
# 3.1 – Garantia dos tipos finais
# ------------------------------------------------------------

df["DataLancamento"] = df["DataLancamento"].dt.date
df["ValorRecebido"] = df["ValorRecebido"].astype(float)

# ------------------------------------------------------------
# 3.2 – Seleção e ordenação das colunas finais
# ------------------------------------------------------------

colunas_final = [
    "DataLancamento",
    "ValorRecebido",
    "CodigoFavorecido",
    "NomeFavorecido",
    "SiglaUF",
    "NomeMunicipio",
    "CodigoOrgaoSuperior",
    "NomeOrgaoSuperior",
    "CodigoOrgao",
    "NomeOrgao",
    "CodigoUnidadeGestora",
    "NomeUnidadeGestora"
]

df = df[colunas_final]

# ------------------------------------------------------------
# 3.3 – Exportação do dataset final
# ------------------------------------------------------------

output_file = os.path.join(
    PROCESSED_DIR,
    "despesas_favorecidos_202511_powerbi.csv"
)

df.to_csv(
    output_file,
    index=False,
    sep=';',
    encoding='latin1'
)

print("🎉 Dataset final preparado para consumo no Power BI!")

🎉 Dataset final preparado para consumo no Power BI!


# ✅ Boas Práticas Aplicadas

* Separação clara entre dados brutos e processados;

* Código organizado por etapas do ETL;

* Tratamento explícito de erros e validações;

* Padronização de nomes e tipos de dados;

* Decisões técnicas baseadas na natureza real dos dados;

* Foco em reprodutibilidade, clareza e qualidade dos dados.

# 🎯 Conclusão

Este projeto demonstra a aplicação prática de um pipeline ETL completo, desde a extração de dados públicos até a entrega de um dataset pronto para análise. A solução equilibra **qualidade técnica, clareza de implementação e boas práticas de engenharia de dados**.